In [8]:
import pdfplumber
from tabulate import tabulate
from functools import *
import re
import fitz

def extract_tables_from_pdf(pdf_path):
    try:
        tables_with_page = []

        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                tables = page.extract_tables()
                for table in tables:
                    tables_with_page.append((page_number, table))

        plain_text_tables = []

        for page_number, table in tables_with_page:
            plain_text = tabulate(table, tablefmt="plain")
            plain_text_tables.append((page_number, plain_text))

        return plain_text_tables

    except Exception as e:
        print(f'An error occurred: {e}')

def clean_text(text):
    # Replace newlines and multiple spaces with a single space
    cleaned_text = re.sub(r'\n+', ' ', text)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def process_tables(tables_with_page, text_by_page):
    print(len(text_by_page))
    for page_number, table in tables_with_page:
        page_index = page_number - 1
        # Check if the page_index is valid
        if page_index >= 0 and page_index < len(text_by_page):
            table_text = clean_text(table)
            page_text = text_by_page[page_index]
            page_text = clean_text(page_text)

            if table_text in page_text:
                page_text = page_text.replace(table_text, '')

            text_by_page[page_index] = page_text
            # print("Page:",page_number)
            # print("\ntable text:",table_text,"\n\n")
            # print("\npage text:",page_text)
            # print("-"*100)

    return text_by_page

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text_by_page = []
    for page_number in range(doc.page_count):
        page = doc[page_number]
        text = page.get_text()
        text_by_page.append(text)
    return text_by_page

def read_pdf_ex_tables(pdf_filename):
    tables_with_page = extract_tables_from_pdf(pdf_filename)
    text_by_page = extract_text_from_pdf(pdf_filename)
    process_tables(tables_with_page,text_by_page)
    pdf_text = ""
    for k,v in enumerate(text_by_page):
        pdf_text += v
    return pdf_text


In [42]:
import pdfplumber
pdf = pdfplumber.open("/content/EXata-8.1.1-ReleaseNotes.pdf")
numPages = len(pdf.pages)
for number, pageText in enumerate(pdf.pages):
      print("Page Number:", number)
      pt=pageText.extract_text()
      pts=pt.split("\n")
      pts=pts[1:-1]
      cleantxt="\n".join(pts)+"\n"
      print(cleantxt)


Page Number: 0
Release Notes

Page Number: 1
© Keysight Technologies, Inc. 2011-2023 U.S. Government Rights Safety Notices
No part of this manual may be reproduced in The Software is "commercial computer
any form or by any means (including software," as defined by Federal Acquisition CAUTION
electronic storage and retrieval or Regulation ("FAR") 2.101. Pursuant to FAR
translation into a foreign language) without 12.212 and 27.405-3 and Department of A CAUTION notice denotes a hazard.
prior agreement and written consent from Defense FAR Supplement ("DFARS") It calls attention to an operating
Keysight Technologies, Inc. as governed by 227.7202, the U.S. government acquires
procedure, practice, or the like that,
United States and international copyright commercial computer software under the
laws. same terms by which the software is if not correctly performed or
customarily provided to the public. adhered to, could result in damage
Revision Accordingly, Keysight provides the Software to t

In [39]:
pdf = pdfplumber.open("/content/EXata-8.1.1-ReleaseNotes.pdf")
numPages = len(pdf.pages)
for number, pageText in enumerate(pdf.pages):
    if number>28:
      print("Page Number:", number)
      pageText= pageText.crop((0.1*pageText.width,0.10*pageText.height,0.9*pageText.width, 0.92*pageText.height))
      pageText.to_image().save("/content/k1.PNG", format="PNG")
      print(pageText.extract_text())
    if number==29:
      break


Page Number: 29
TABLE 31. Limitations and Known Issues for EXata 8.0.2 (Continued)
Feature Description
Scenario Player The Network Scanning and Port Scanning attacks can not be stopped from the
Attack History Manager. They must be stopped from the HITL interface.
When running EXata in remote execution mode from the Scenario Player, stopping
the simulation does not always stop the simulator. In such cases, the EXata
process has to be stopped manually.
In Signal Intelligence attack, verbose should be disabled when executing from the
Scenario Player, since it generates a lot of simulator output logs leading to missing
some events. Instead, the Stats DB can be used for getting the signal log.
Adaptive Attack Script The standard error stream is not printed to the error log. Only the standard output
stream is printed to the log.
Scenario Analyzer Heatmaps are currently only supported for Lat-Lon-Alt scenarios. Cartesian
scenarios are not supported.
Real-time network statistics currently show